In [57]:
#The Van der Pol oscillator
#  ddx - mu*(1 - x**2)*dx + x = A*cos(w*t)

In [88]:
%run Nonlinear_Dynamics
def solve_sdof(max_time=10.0, mu = 8.3, A = 1.2, w = 2*np.pi/10, x0 = 0, v0 = 0, plotnow = 1):

    
    def sdof_deriv(x1_x2, t, A = 1.2, w = 2*np.pi/10, mu = 8.3):
        """Compute the time-derivative of a SDOF system."""
        x1, x2 = x1_x2
        return [x2, mu*(1 - x1**2)*x2 - x1 + A*np.cos(w*t)]

    x0i=((x0, v0))
    # Solve for the trajectories
    t = sp.linspace(0, max_time, int(250*max_time))
    x_t = sp.integrate.odeint(sdof_deriv, x0i, t)
    
    x, v = x_t.T
    f = A*np.cos(w*t)
    
    if plotnow == 1:
        #fig = plt.figure()
        #ax = fig.add_axes([0, 0, 1, 1], projection='3d')
        plt.plot(t,x,t,f,'--')
        pl.legend(['x','Forcing Function'])
        plt.xlabel('Time (s)')
        plt.ylabel('x')
        plt.title('Integrated Response of the Forced Van der Pol Oscillator')
        plt.show()

    return t, x, v


In [90]:
solve_sdof(max_time=30.0, x0 = 0, v0 = 0, plotnow = 1)

(array([  0.00000000e+00,   4.00053340e-03,   8.00106681e-03, ...,
          2.99919989e+01,   2.99959995e+01,   3.00000000e+01]),
 array([  0.00000000e+00,   9.71061679e-06,   3.92741243e-05, ...,
          1.94110831e+00,   1.94097909e+00,   1.94084988e+00]),
 array([ 0.        ,  0.00488122,  0.00992711, ..., -0.03230308,
        -0.03229926, -0.03229578]))

In [86]:
#From nonlinear.py posted by Daniel Clark
import pylab as pl
import numpy as np
import scipy.optimize as sci
import scipy.integrate as sp

# this code is the nonlinear case of \dotdot{x} - mu*(1-x**2)\dot{x} + x = A*cos(w*t)

def VanDerPolOscillatorTimeSeriesResults(N = 99,mu = 8.53,w = 2*np.pi/10,A = 1.2):
    time = np.linspace(0, 10*np.pi, N+1)    # time samples of forcing function
    time = time[0:-1]                         # Removing the extra sample
    f = A*np.cos(w*time)                     # My forcing function
    T = time[-1]
    xbar = 1.8/1.2*f#A*np.cos(1.5*time)

    def FUNCTION(xbar):
        N = len(xbar)
        Xbar = np.fft.fft(xbar)
        omega = np.fft.fftfreq(N, T/(2*np.pi*N) )# + 0.0000001 # list of frequencies
        dotxbar = np.fft.ifft(np.multiply((1j*omega),Xbar))
        dotdotxbar = np.fft.ifft(np.multiply((1j*omega)**2,Xbar))
        R = dotdotxbar - mu*(1-xbar**2)*dotxbar + xbar - f
        R = R**2
        R = np.sum(R)
        return R

    optimizedResults = sci.minimize(FUNCTION, xbar, method='SLSQP')
    xbar = optimizedResults.x

    #print(optimizedResults)
    #print(xbar)

    pl.plot(time,xbar,time,f,'--')
    pl.legend(['x','Forcing Function'])
    pl.xlabel('Time (s)')
    pl.title('Harmonic Balance Method Response of the Van der Pol Oscillator')
    pl.show()
    return time, xbar

In [87]:
VanDerPolOscillatorTimeSeriesResults(N = 400)

C:\Users\tidba_000\Anaconda3\lib\site-packages\scipy\optimize\slsqp.py:62: ComplexWarning: Casting complex values to real discards the imaginary part
  jac[i] = (func(*((x0+dx,)+args)) - f0)/epsilon
C:\Users\tidba_000\Anaconda3\lib\site-packages\scipy\optimize\slsqp.py:407: ComplexWarning: Casting complex values to real discards the imaginary part
  slsqp(m, meq, x, xl, xu, fx, c, g, a, acc, majiter, mode, w, jw)


(array([  0.        ,   0.07853982,   0.15707963,   0.23561945,
          0.31415927,   0.39269908,   0.4712389 ,   0.54977871,
          0.62831853,   0.70685835,   0.78539816,   0.86393798,
          0.9424778 ,   1.02101761,   1.09955743,   1.17809725,
          1.25663706,   1.33517688,   1.41371669,   1.49225651,
          1.57079633,   1.64933614,   1.72787596,   1.80641578,
          1.88495559,   1.96349541,   2.04203522,   2.12057504,
          2.19911486,   2.27765467,   2.35619449,   2.43473431,
          2.51327412,   2.59181394,   2.67035376,   2.74889357,
          2.82743339,   2.9059732 ,   2.98451302,   3.06305284,
          3.14159265,   3.22013247,   3.29867229,   3.3772121 ,
          3.45575192,   3.53429174,   3.61283155,   3.69137137,
          3.76991118,   3.848451  ,   3.92699082,   4.00553063,
          4.08407045,   4.16261027,   4.24115008,   4.3196899 ,
          4.39822972,   4.47676953,   4.55530935,   4.63384916,
          4.71238898,   4.7909288 ,   4.

In [82]:
from scipy.signal import correlate
#help(correlate)

Help on function correlate in module scipy.signal.signaltools:

correlate(in1, in2, mode='full')
    Cross-correlate two N-dimensional arrays.
    
    Cross-correlate `in1` and `in2`, with the output size determined by the
    `mode` argument.
    
    Parameters
    ----------
    in1 : array_like
        First input.
    in2 : array_like
        Second input. Should have the same number of dimensions as `in1`;
        if sizes of `in1` and `in2` are not equal then `in1` has to be the
        larger array.
    mode : str {'full', 'valid', 'same'}, optional
        A string indicating the size of the output:
    
        ``full``
           The output is the full discrete linear cross-correlation
           of the inputs. (Default)
        ``valid``
           The output consists only of those elements that do not
           rely on the zero-padding.
        ``same``
           The output is the same size as `in1`, centered
           with respect to the 'full' output.
    
    Return